In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima

In [2]:
file_path = r"C:\Code\energy-demand-forecasting\data\raw\use_all_btu.csv"

data = pd.read_csv(file_path)

In [ ]:
print("First 5 rows of the dataset:")
display(data.head())

In [ ]:
print("\nDataset Info:")
data.info()

In [ ]:
print("\nMissing Values in Each Column:")
print(data.isnull().sum())

In [ ]:
print("\nMissing Values in Each Column:")
print(data.isnull().sum())

# Observations from the Data

## Structure:

- The dataset has 14,773 rows and 66 columns.
- The columns Data_Status, State, and MSN are categorical, while the rest are numeric years (1960–2022).

## Missing Data:

- The columns for earlier years (e.g., 1960–1969) have significant missing values (4264 out of 14,773 rows).
- Recent years (2018–2022) have no missing values.

## Potentially Useful Columns

- Data_Status: Indicates the status of the data (e.g., final forecast).
- State: State-wise energy consumption data.
- MSN: Codes for energy types or activities (must decode using msn_codes_and_descriptions.xlsx).

## Time-Series Structure:

- Data is structured as a time series, with rows representing State and MSN, and columns being years (1960–2022).

In [ ]:
data_long = pd.melt(
    data, 
    id_vars=['Data_Status', 'State', 'MSN'], 
    var_name='Year', 
    value_name='Energy_Consumption'
)

data_long['Year'] = data_long['Year'].astype(int)

print(data_long.head())


In [ ]:
msn_codes = pd.read_excel(r'C:\Code\energy-demand-forecasting\docs\msn_codes_and_descriptions.xlsx')
msn_codes_path = r"C:\Code\energy-demand-forecasting\docs\msn_codes_and_descriptions.xlsx"
msn_codes_df = pd.read_excel(msn_codes_path)

# Display the first few rows of the MSN codes dataframe
msn_codes_df.head()

In [ ]:
msn_codes_sheets = pd.ExcelFile(msn_codes_path).sheet_names
msn_codes_sheets

In [ ]:
# Load the 'MSN descriptions' sheet
msn_descriptions_df = pd.read_excel(msn_codes_path, sheet_name='MSN descriptions')

# Display the first few rows of the loaded sheet to confirm the structure
msn_descriptions_df.head()

In [ ]:
msn_descriptions_updated_df = pd.read_excel(msn_codes_path, sheet_name='MSN descriptions')

# Display the first few rows of the updated file to confirm the structure
msn_descriptions_updated_df.head()

In [ ]:
# Clean the MSN descriptions table
msn_cleaned_df = msn_descriptions_updated_df[['MSN', 'Description', 'Unit']].dropna().reset_index(drop=True)

# Display the cleaned table
msn_cleaned_df.head()

In [ ]:
# Load the main dataset
use_all_btu_path = r"C:\Code\energy-demand-forecasting\data\raw\use_all_btu.csv"
use_all_btu_df = pd.read_csv(use_all_btu_path)

# Merge the main dataset with the MSN descriptions
merged_df = pd.merge(use_all_btu_df, msn_cleaned_df, on="MSN", how="left")

# Display the first few rows of the merged dataset
print(merged_df.head())

# Save the merged dataset for future use
merged_df.to_csv(r"C:\Code\energy-demand-forecasting\data\processed\use_all_btu_with_descriptions.csv", index=False)


In [ ]:
# Define the path to the main dataset
use_all_btu_path = r"C:\Code\energy-demand-forecasting\data\raw\use_all_btu.csv"

# Load the main dataset
use_all_btu_df = pd.read_csv(use_all_btu_path)

# Merge the main dataset with the MSN descriptions
merged_df = pd.merge(use_all_btu_df, msn_cleaned_df, on="MSN", how="left")

# Save the merged dataset for future use
processed_path = r"C:\Code\energy-demand-forecasting\data\processed\use_all_btu_with_descriptions.csv"
merged_df.to_csv(processed_path, index=False)

# Display the first few rows of the merged dataset
merged_df.head(), processed_path


In [15]:
# Filter for nuclear-related MSNs
nuclear_data = merged_df[merged_df['Description'].str.contains('nuclear', case=False, na=False)]


In [ ]:
print("Nuclear-Related Data:")
display(nuclear_data.head())

In [17]:
# Reshape data to long format for analysis
nuclear_long = pd.melt(
    nuclear_data,
    id_vars=['Data_Status', 'State', 'MSN', 'Description', 'Unit'],
    var_name='Year',
    value_name='Energy_Consumption'
)

In [18]:
nuclear_long['Year'] = pd.to_numeric(nuclear_long['Year'], errors='coerce')

In [19]:
nuclear_long = nuclear_long.dropna(subset=['Energy_Consumption'])

In [ ]:
state_data = nuclear_long[nuclear_long['State'] == 'CA']
plt.figure(figsize=(10, 6))
for msn in state_data['MSN'].unique():
    subset = state_data[state_data['MSN'] == msn]
    plt.plot(subset['Year'], subset['Energy_Consumption'], label=msn)

plt.title("Nuclear Energy Consumption in California")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend(title="MSN")
plt.grid()
plt.show()

In [ ]:
# Focus on the MSN code 'NUEGB' for detailed analysis
nuegb_data = nuclear_long[(nuclear_long['State'] == 'CA') & (nuclear_long['MSN'] == 'NUEGB')]

# Plot the trend for NUEGB
plt.figure(figsize=(10, 6))
plt.plot(nuegb_data['Year'], nuegb_data['Energy_Consumption'], marker='o', label='NUEGB')

# Highlight drop-off points
plt.axvline(x=2010, color='red', linestyle='--', label='Drop-off Start')
plt.title("Nuclear Energy Consumption (NUEGB) in California")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Focus on NUEGB across all states
nuegb_all_states = nuclear_long[nuclear_long['MSN'] == 'NUEGB']

# Plot trends for the top 5 states with the most data
top_states = nuegb_all_states['State'].value_counts().head(5).index
plt.figure(figsize=(12, 8))
for state in top_states:
    state_data = nuegb_all_states[nuegb_all_states['State'] == state]
    plt.plot(state_data['Year'], state_data['Energy_Consumption'], label=state)

plt.title("Nuclear Energy Consumption (NUEGB) Across Top 5 States")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend(title="State")
plt.grid()
plt.show()


In [ ]:
# Identify years with missing or zero values
missing_years = nuegb_data[nuegb_data['Energy_Consumption'] == 0]
print("Years with missing or zero consumption in California:")
print(missing_years)


In [ ]:
# Sum nuclear energy consumption across all states for NUEGB
national_trends = nuclear_long[nuclear_long['MSN'] == 'NUEGB'].groupby('Year').sum()

# Plot national trends
plt.figure(figsize=(12, 8))
plt.plot(national_trends.index, national_trends['Energy_Consumption'], marker='o', color='blue', label='National Total')
plt.title("National Nuclear Energy Consumption Trends (NUEGB)")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Filter nuclear energy data for Texas (state code: TX)
state_code = 'TX'  # Replace 'TX' with the desired state code
state_data = nuclear_long[(nuclear_long['State'] == state_code) & (nuclear_long['MSN'] == 'NUEGB')]

# Plot trends for Texas
plt.figure(figsize=(12, 8))
plt.plot(state_data['Year'], state_data['Energy_Consumption'], marker='o', color='green', label=f'{state_code}')
plt.title(f"Nuclear Energy Consumption in {state_code}")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Compare trends for multiple states (e.g., CA, TX, NY, FL, IL, PA, SC)
selected_states = ['CA', 'TX', 'NY', 'FL', 'IL', 'PA', 'SC']
plt.figure(figsize=(12, 8))
for state in selected_states:
    state_data = nuclear_long[(nuclear_long['State'] == state) & (nuclear_long['MSN'] == 'NUEGB')]
    plt.plot(state_data['Year'], state_data['Energy_Consumption'], label=state)

plt.title("Nuclear Energy Consumption Trends Across Selected States")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend(title="State")
plt.grid()
plt.show()


In [ ]:
# Total consumption by state for NUEGB
state_totals = nuclear_long[nuclear_long['MSN'] == 'NUEGB'].groupby('State')['Energy_Consumption'].sum().sort_values(ascending=False)
print("Top states for nuclear energy consumption:")
print(state_totals.head())


In [ ]:
# Recreate the nuclear_long DataFrame
nuclear_long = pd.melt(
    merged_df[merged_df['Description'].str.contains('nuclear', case=False, na=False)],
    id_vars=['Data_Status', 'State', 'MSN', 'Description', 'Unit'],
    var_name='Year',
    value_name='Energy_Consumption'
)

# Ensure Year is numeric
nuclear_long['Year'] = pd.to_numeric(nuclear_long['Year'], errors='coerce')

# Drop rows with missing values in Energy_Consumption
nuclear_long = nuclear_long.dropna(subset=['Energy_Consumption'])

# Identify the top 5 states by total nuclear energy consumption
top_states = (
    nuclear_long[nuclear_long['MSN'] == 'NUEGB']
    .groupby('State')['Energy_Consumption']
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

# Plot trends for the top 5 states
plt.figure(figsize=(12, 8))
for state in top_states:
    state_data = nuclear_long[(nuclear_long['State'] == state) & (nuclear_long['MSN'] == 'NUEGB')]
    plt.plot(state_data['Year'], state_data['Energy_Consumption'], marker='o', label=state)

plt.title("Nuclear Energy Consumption Trends for Top 5 States")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend(title="State")
plt.grid()
plt.show()


In [ ]:
# Focus on national-level nuclear energy data for NUEGB
national_data = nuclear_long[nuclear_long['State'] == 'US']  # US is often used for national aggregates

# Ensure data is sorted by year
national_data = national_data[national_data['MSN'] == 'NUEGB'].sort_values('Year')

# Keep only Year and Energy_Consumption for simplicity
forecast_data = national_data[['Year', 'Energy_Consumption']].set_index('Year')

# Display the data to confirm structure
print(forecast_data.head())


In [ ]:
# Filter for national-level data starting from 1960
national_data_1960 = nuclear_long[
    (nuclear_long['State'] == 'US') & 
    (nuclear_long['Year'] >= 1960) & 
    (nuclear_long['MSN'] == 'NUEGB')
].sort_values('Year')

# Keep only Year and Energy_Consumption
forecast_data_1960 = national_data_1960[['Year', 'Energy_Consumption']].set_index('Year')

# Display the filtered data
print(forecast_data_1960.head())


In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt

# Train-test split
train_1960 = forecast_data_1960.iloc[:-5]  # Use all but the last 5 years as training
test_1960 = forecast_data_1960.iloc[-5:]   # Use the last 5 years as testing

# Apply Exponential Smoothing
model_1960 = ExponentialSmoothing(train_1960, trend="add", seasonal=None, damped_trend=True).fit()

# Forecast the next 5 years
forecast_1960 = model_1960.forecast(steps=5)

# Correct Year handling and plot
plt.figure(figsize=(10, 6))
plt.plot(train_1960.index, train_1960['Energy_Consumption'], label='Training Data')
plt.plot(test_1960.index, test_1960['Energy_Consumption'], label='Actual Data', color='orange')
plt.plot(test_1960.index, forecast_1960, label='Forecast', color='green', linestyle='--')
plt.title("Nuclear Energy Consumption Forecast (National, Starting 1960)")
plt.xlabel("Year")
plt.xticks(train_1960.index[::5], rotation=45)  # Adjust year ticks for readability
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend()
plt.grid()
plt.show()




In [ ]:
mae_1960 = mean_absolute_error(test_1960, forecast_1960)
rmse_1960 = np.sqrt(mean_squared_error(test_1960, forecast_1960))

print(f"Mean Absolute Error (MAE): {mae_1960}")
print(f"Root Mean Square Error (RMSE): {rmse_1960}")



In [ ]:
# Automatically find the best ARIMA parameters for the training data
auto_arima_model = auto_arima(
    train_1960,  # Training data
    seasonal=False,  # Assume no seasonality
    stepwise=True,  # Use a stepwise search to find the best parameters
    suppress_warnings=True,  # Suppress warnings for better readability
    error_action="ignore",  # Ignore errors for edge cases
    trace=True  # Show the parameter search process
)

# Forecast the next 5 years
auto_arima_forecast = auto_arima_model.predict(n_periods=5)

# Plot the forecast
plt.figure(figsize=(10, 6))
plt.plot(train_1960, label="Training Data")
plt.plot(test_1960, label="Actual Data", color="orange")
plt.plot(test_1960.index, auto_arima_forecast, label="Auto ARIMA Forecast", color="green", linestyle="--")
plt.title("Nuclear Energy Consumption Forecast with Auto ARIMA")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend()
plt.grid()
plt.show()

# Print the summary of the Auto ARIMA model
auto_arima_model.summary()


In [34]:
# Create lagged features for supervised learning
def create_lagged_features(data, lags=3):
    df = data.copy()
    for lag in range(1, lags + 1):
        df[f"Lag_{lag}"] = df['Energy_Consumption'].shift(lag)
    df.dropna(inplace=True)  # Remove rows with NaN values caused by lagging
    return df

# Prepare the dataset
ml_data = create_lagged_features(forecast_data_1960, lags=3)

# Split into training and testing sets
X_train = ml_data.iloc[:-5, 1:]  # Lagged features
y_train = ml_data.iloc[:-5, 0]   # Target variable
X_test = ml_data.iloc[-5:, 1:]
y_test = ml_data.iloc[-5:, 0]


In [ ]:
# Initialize and train the XGBoost model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions
xgb_forecast = xgb_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, xgb_forecast)
rmse = np.sqrt(mean_squared_error(y_test, xgb_forecast))
print(f"XGBoost - MAE: {mae}, RMSE: {rmse}")

# Plot the results
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(ml_data.index[:-5], y_train, label="Training Data")
plt.plot(ml_data.index[-5:], y_test, label="Actual Data", color="orange")
plt.plot(ml_data.index[-5:], xgb_forecast, label="XGBoost Forecast", color="green", linestyle="--")
plt.title("Nuclear Energy Consumption Forecast with XGBoost")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
rf_forecast = rf_model.predict(X_test)

# Evaluate the model
mae_rf = mean_absolute_error(y_test, rf_forecast)
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_forecast))
print(f"Random Forest - MAE: {mae_rf}, RMSE: {rmse_rf}")

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(ml_data.index[:-5], y_train, label="Training Data")
plt.plot(ml_data.index[-5:], y_test, label="Actual Data", color="orange")
plt.plot(ml_data.index[-5:], rf_forecast, label="Random Forest Forecast", color="blue", linestyle="--")
plt.title("Nuclear Energy Consumption Forecast with Random Forest")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Recreate national_data_1960 to define the dataset properly
national_data_1960 = nuclear_long[
    (nuclear_long['State'] == 'FL') & 
    (nuclear_long['Year'] >= 1960) & 
    (nuclear_long['MSN'] == 'NUEGB')
].sort_values('Year')

# Extract only the necessary columns
forecast_data_1960 = national_data_1960[['Year', 'Energy_Consumption']].set_index('Year')

# Recreate the lagged features and split the data
ml_data = create_lagged_features(forecast_data_1960, lags=3)

# Split into training and testing sets
X_train = ml_data.iloc[:-5, 1:]  # Lagged features
y_train = ml_data.iloc[:-5, 0]   # Target variable
X_test = ml_data.iloc[-5:, 1:]
y_test = ml_data.iloc[-5:, 0]

# Retrain and compare XGBoost and Random Forest models
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
xgb_model.fit(X_train, y_train)

# Forecast with XGBoost
xgb_forecast = xgb_model.predict(X_test)

# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Forecast with Random Forest
rf_forecast = rf_model.predict(X_test)

# Evaluate models
xgb_mae = mean_absolute_error(y_test, xgb_forecast)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_forecast))
rf_mae = mean_absolute_error(y_test, rf_forecast)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_forecast))

print(f"XGBoost - MAE: {xgb_mae}, RMSE: {xgb_rmse}")
print(f"Random Forest - MAE: {rf_mae}, RMSE: {rf_rmse}")

# Plot results for comparison
plt.figure(figsize=(10, 6))
plt.plot(ml_data.index[:-5], y_train, label="Training Data")
plt.plot(ml_data.index[-5:], y_test, label="Actual Data", color="orange")
plt.plot(ml_data.index[-5:], xgb_forecast, label="XGBoost Forecast", color="green", linestyle="--")
plt.plot(ml_data.index[-5:], rf_forecast, label="Random Forest Forecast", color="blue", linestyle="--")
plt.title("Nuclear Energy Consumption Forecast: XGBoost vs Random Forest")
plt.xlabel("Year")
plt.ylabel("Energy Consumption (Billion BTU)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Calculate Mean Absolute Percentage Error (MAPE) for XGBoost and Random Forest
mape_xgb = (xgb_mae / y_test.mean()) * 100
mape_rf = (rf_mae / y_test.mean()) * 100

# Print MAPE results
print(f"XGBoost MAPE: {mape_xgb:.2f}%")
print(f"Random Forest MAPE: {mape_rf:.2f}%")


In [ ]:
# Recreate necessary functions and data
def create_lagged_features(data, lags=3):
    df = data.copy()
    for lag in range(1, lags + 1):
        df[f"Lag_{lag}"] = df['Energy_Consumption'].shift(lag)
    df.dropna(inplace=True)
    return df

# Placeholder for forecast_data_1960, replace with actual data loading code
# Simulated dummy data for context since the environment was reset
years = pd.date_range(start='1960', periods=62, freq='Y')
energy_consumption = np.cumsum(np.random.randint(100, 200, size=62))  # Simulated growth data
forecast_data_1960 = pd.DataFrame({'Energy_Consumption': energy_consumption}, index=years.year)

# Prepare the data for machine learning
ml_data = create_lagged_features(forecast_data_1960, lags=3)

# Split into training and testing sets
X_train = ml_data.iloc[:-5, 1:]
y_train = ml_data.iloc[:-5, 0]
X_test = ml_data.iloc[-5:, 1:]
y_test = ml_data.iloc[-5:, 0]

# Train XGBoost model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_forecast = xgb_model.predict(X_test)

# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_forecast = rf_model.predict(X_test)

# Calculate MAE for both models
xgb_mae = mean_absolute_error(y_test, xgb_forecast)
rf_mae = mean_absolute_error(y_test, rf_forecast)

# Calculate MAPE for XGBoost and Random Forest
mape_xgb = (xgb_mae / y_test.mean()) * 100
mape_rf = (rf_mae / y_test.mean()) * 100

# Print MAPE results
print(f"XGBoost MAPE: {mape_xgb:.2f}%")
print(f"Random Forest MAPE: {mape_rf:.2f}%")
